In [28]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

data = pd.read_csv('Coffee-Shop-Sales.csv')
data['total_price'] = data['transaction_qty'] * data['unit_price']
datetime_format = '%H:%M:%S'
data['transaction_time'] = pd.to_datetime(data['transaction_time'], format=datetime_format)


In [29]:
# Konversi kolom transaction_date ke tipe data datetime
data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='mixed')

# Tentukan rentang waktu dalam format yang sesuai
start_date = '2023-01-01' 
end_date = '2023-01-31' 

# Filter berdasarkan wilayah dan rentang waktu
data_in_month = data[(data['transaction_date'] >= start_date) & (data['transaction_date'] <= end_date)]


In [30]:
# Misalnya data_in_month adalah DataFrame yang berisi data penjualan harian pada bulan Januari
# Anda dapat menggantinya dengan data dari dataset Anda
data_in_month = data[(data['transaction_date'] >= '2023-01-01') & (data['transaction_date'] <= '2023-01-31')]

# Mengelompokkan data berdasarkan tanggal dan lokasi toko, dan menghitung total pendapatan per hari
daily_revenue = data_in_month.groupby(['transaction_date', 'store_location'])['total_price'].sum().unstack().reset_index()

# Plot menggunakan Plotly Express
fig = px.line(daily_revenue, x='transaction_date', y=daily_revenue.columns[1:], 
              title='Pendapatan Harian di Semua Wilayah - Januari',
              labels={'value': 'Pendapatan', 'transaction_date': 'Tanggal', 'variable': 'Lokasi Toko'})

# Tampilkan plot
fig.show()


In [31]:
# Mengelompokkan hasilnya berdasarkan wilayah dan bulan, dan menghitung total pendapatan per bulan
monthly_revenue = data.groupby([data['store_location'], data['transaction_date'].dt.to_period('M')])['total_price'].sum().reset_index()

# Mengonversi objek Period menjadi string dengan format tahun-bulan
monthly_revenue['Bulan'] = monthly_revenue['transaction_date'].dt.strftime('%Y-%m')

# Renamakan kolom agar sesuai dengan kebutuhan
monthly_revenue = monthly_revenue.rename(columns={'store_location': 'Lokasi toko'})

# Plot menggunakan Plotly Express dengan barmode='group' untuk bar plot yang tidak ditumpuk
fig = px.bar(monthly_revenue, x='Bulan', y='total_price', color='Lokasi toko', 
             title='Pendapatan per Bulan di setiap wilayah',
             labels={'total_price': 'Pendapatan', 'Bulan': 'Bulan', 'Lokasi toko': 'Lokasi Toko'},
             barmode='group')

# Tampilkan plot
fig.show()


In [32]:
# total pendapatan keseluruhan di semua wilayah
total_revenue_by_location = data.groupby('store_location')['total_price'].sum().reset_index()

# Membuat plot pie menggunakan Plotly Express
fig = px.pie(total_revenue_by_location, names='store_location', values='total_price', title='Total Pendapatan Berdasarkan Lokasi Toko', labels={'store_location': 'Lokasi', 'total_price': 'Pendapatan'})

# Menampilkan plot
fig.show()




In [47]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Filter data untuk bulan Januari dan wilayah tertentu
january_data = pd.DataFrame(data[(data['transaction_date'].dt.month == 1) & 
                    (data['store_location'].isin(['Astoria', 'Lower Manhattan', "Hell's Kitchen"]))])

# Sisipkan baris ini di sini
bins = [6, 12, 17, 18, 24]
labels = ['Pagi', 'Siang', 'Sore', 'Malam']
january_data.loc[:, 'time_category'] = pd.cut(january_data['transaction_time'].dt.hour, bins=bins, labels=labels, right=False)

# Melakukan pengelompokan data
visitor_data = january_data.groupby(['store_location', 'time_category'], observed=False).agg({'total_price': 'sum', 'transaction_id': 'count'}).reset_index()
visitor_data.columns = ['store_location', 'time_category', 'total_price', 'visitor_count']

# Membuat subplot
fig = make_subplots(rows=1, cols=3, subplot_titles=['Astoria', 'Lower Manhattan', "Hell's Kitchen"], specs=[[{'type':'pie'}, {'type':'pie'}, {'type':'pie'}]])

# Loop untuk menambahkan pie chart pada setiap subplot
for i, location in enumerate(['Astoria', 'Lower Manhattan', "Hell's Kitchen"], start=1):
    location_data = visitor_data[visitor_data['store_location'] == location]
    fig.add_trace(go.Pie(labels=location_data['time_category'], values=location_data['visitor_count'],
                         hovertemplate='<b>Waktu</b>: %{label}<br><b>Pengunjung</b>: %{value}<br><b>Pendapatan</b>: %{text}',
                         text=location_data['total_price'], name=location),
                  row=1, col=i)

# Update layout
fig.update_layout(title='Analisis Pengunjung dan Pendapatan Berdasarkan Waktu - Januari',
                  showlegend=True)
fig.show()


